In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
import os
import cv2
from tqdm.auto import tqdm
import argparse
import pandas as pd
from torch.utils.data.dataset import Dataset

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class myDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        # self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.img_lists = sorted(os.listdir(self.img_dir))
        # print(self.img_lists)
    def __len__(self):
        return len(self.img_lists)

    def __getitem__(self, idx):
        fileName = self.img_lists[idx]
        img_path = os.path.join(self.img_dir, fileName)
        image = Image.open(img_path)
        label = fileName[:-4].split('_')[0]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = (self.target_transform(label))
        # return imageName, image, label
        return  (fileName, image, int(label))

In [ ]:
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
test_dataset = myDataset(img_dir='hw1_data/p1_data/val_50',transform=test_tfm)
dataloader = DataLoader(test_dataset, batch_size=128,shuffle=False, num_workers=0)

In [ ]:
outfile_dir = 'p1_result.csv'

model_path = 'save_models/64_50_vgg13_bn_cosineAnneal_changedata.pt'

model = torchvision.models.vgg13_bn(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs,50)

model.load_state_dict(torch.load(model_path))
model = model.to(device)
# print(model)
model.eval()

In [ ]:
features = []
def hook(module, input, output):
    features.append(output.clone().detach())

handle = model.classifier[-2].register_forward_hook(hook)

all_val = torch.empty((0,4096))
targets = np.zeros((0,))
for imgNames, imgs, labels in (dataloader):
    labels = labels.detach().numpy()
    # print(labels.shape)
    targets = np.concatenate((labels,targets),axis=0)
    with torch.no_grad():
        logits = model(imgs.to(device))
    all_val = torch.cat((all_val.cpu(),features[-1].cpu()),0)

layerOut = all_val.detach().numpy()

print(layerOut.shape,layerOut[0].shape)
print(targets.shape,targets[0])
# for f in features:
#     all_val = torch.cat((all_val,f),0)
handle.remove()


In [ ]:
from sklearn.manifold import TSNE
embedded = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(layerOut)
print(embedded.shape)
print(embedded[0])

from matplotlib import pyplot as plt
from itertools import cycle
cycol = cycle('bgrcmk')

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

cmap = get_cmap(50)
plt.figure(figsize=(16, 10))
for label in range(50):
    index = []
    for i, t in enumerate(targets):
        if t == label:
            index.append(i)
    plt.scatter(embedded[index,0], embedded[index,1], c=cmap(label), label=label) #next(cycol)
plt.legend()
plt.show()
